### Mixup Test - proof of concept (doesn't work)

If we split our model into 2 parts embedding + rest_of_the_model, then we can use the second part as a model and just shift the input of it (in dataloader or callback). We just pass out initial data though embeddings layer and then blend the result. That will be our inputs. I think it’s fair to call in this case the second part (rest_of_the_model) as ‘the model’ as only this part can be trained (I cannot think of the way how to train embeddings as well in a mixup). And feedforward though embeddings is now just a part of preprocessing step. Definetly, first of all we have to train your model in a normal way, as we want to produce our embeddings. Then we can use the_rest_of_the_model and retrain it or throw it away and use only embeddings (and new the_rest_of_the_model) for a mixup training.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.tabular import *

In [3]:
from exp.nb_ import *

In [4]:
path=Path('data/')
train_df = pd.read_pickle(path/'train_clean.zip', compression='zip')
test_df = pd.read_pickle(path/'test_clean')

In [5]:
procs=[FillMissing, Categorify, Normalize]

In [6]:
cat_vars_tpl = ('Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw')

cont_vars_tpl = ('CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday')
cat_vars = list(cat_vars_tpl)
cont_vars = list(cont_vars_tpl)
all_vars = cat_vars + cont_vars

In [7]:
dep_var = 'Sales'
df = train_df[cat_vars + cont_vars + [dep_var,'Date']].copy()

In [8]:
#this step reduces the data as whole dataset doesn't fit into my memory after preprocessing
np.random.seed(1001)
coef = 0.3
ln = len(df)
part_idx = np.random.choice(ln, int(ln*coef), replace=False)
df = df.iloc[part_idx]
df.sort_values(by='Date', ascending=False, inplace=True)
df = df.reset_index()

In [11]:
cut = df['Date'][(df['Date'] == df['Date'][int(len(test_df)*coef)])].index.max()
cut

12443

In [12]:
valid_idx = range(cut)
valid_idx

range(0, 12443)

In [13]:
BS = 1024

In [14]:
data = (TabularList.from_df(df, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
             .split_by_idx(valid_idx)
             .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
             .databunch(bs=BS))

In [15]:
max_log_y = np.log(np.max(train_df['Sales'])*1.2)
y_range = torch.tensor([0, max_log_y], device=defaults.device)

In [16]:
np.random.seed(1001)
learn = tabular_learner(data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

In [17]:
learn.fit_one_cycle(6, 1e-2, wd=0.2)

epoch,train_loss,valid_loss,exp_rmspe,time
0,0.231496,0.091658,0.375547,00:07
1,0.026047,0.018567,0.139769,00:06
2,0.018347,0.020124,0.153870,00:06
3,0.015385,0.022024,0.162026,00:06
4,0.011140,0.012428,0.113320,00:06
5,0.008664,0.012298,0.108537,00:06


In [18]:
learn.fit_one_cycle(6, 1e-4, wd=0.2)

epoch,train_loss,valid_loss,exp_rmspe,time
0,0.008030,0.012363,0.108429,00:06
1,0.007976,0.012201,0.108119,00:06
2,0.007839,0.012210,0.107686,00:06
3,0.007839,0.012034,0.107568,00:06
4,0.007604,0.012183,0.108370,00:06
5,0.007520,0.012203,0.108599,00:06


### Santity test

Here we try to separate embeddings from the model. First of all we preprocess the data (normalize, categorize and fill missing if needed) Then we take a model and feed it all our data (train and valid in separate dataframes) to embedding layers only. Then we concat these values with cont values. So now we have a bunch of floats for each row of data. This is what our NN (apart from embeddings) really gets as input. Our last step is to pretend that these floats are just a bunch of cont values and try to teach NN in a normal way (without preprocessing (!)) If we will get the similar results as in normal training, then our methon does work and we can think of a mixup.

By the way we already have a function that makes all the preprocess and outputs the 'real model input'. I've made it for Random Forrest with embedding case (RF vs NN) in https://github.com/Pak911/fastai-shared-notebooks/blob/master/interpret_tabular.ipynb

In [57]:
ln = len(df)
train_idx = list_diff(list1=range(ln), list2=valid_idx)

In [58]:
tr_df = df.iloc[train_idx]
val_df = df.iloc[valid_idx]

In [59]:
tr_data_inner = to_np(get_inner_repr(df=tr_df[all_vars], learn=learn))

In [60]:
val_data_inner = to_np(get_inner_repr(df=val_df[all_vars], learn=learn))

In [61]:
len(tr_data_inner), len(val_data_inner)

(240858, 12443)

In [62]:
tr_data_inner[0]

array([-0.041985, -0.018466, -0.071389, -0.03445 , ..., -0.12327 ,  0.193699,  0.361726, -0.061271], dtype=float32)

In [63]:
len(tr_data_inner[0]), len(val_data_inner[0])

(233, 233)

In [64]:
tr_inner_df = pd.DataFrame(tr_data_inner)
val_inner_df = pd.DataFrame(val_data_inner)

In [65]:
tr_inner_df[dep_var] = tr_df.reset_index()[dep_var]
val_inner_df[dep_var] = val_df.reset_index()[dep_var]

In [66]:
merge_inner_df = pd.concat([tr_inner_df, val_inner_df])

In [67]:
inner_val_idx = range(len(tr_inner_df), len(merge_inner_df))

In [68]:
inner_cont_vars = list_diff(merge_inner_df.columns, [dep_var])

In [69]:
inner_procs=[]

In [70]:
inner_data = (TabularList.from_df(merge_inner_df, path=path, cat_names=[], cont_names=inner_cont_vars, procs=inner_procs)
             .split_by_idx(inner_val_idx)
             .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
             .databunch(bs=BS))

In [71]:
np.random.seed(1001)
inner_learn = tabular_learner(inner_data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

In [72]:
inner_learn.fit_one_cycle(6, 1e-2, wd=0.2)

epoch,train_loss,valid_loss,exp_rmspe,time
0,0.224578,0.023641,0.158642,00:07
1,0.019842,0.031679,0.186901,00:06
2,0.014821,0.021796,0.162090,00:06
3,0.011455,0.013880,0.113809,00:07
4,0.009395,0.012489,0.108613,00:07
5,0.007642,0.011904,0.107223,00:07


In [73]:
inner_learn.fit_one_cycle(6, 1e-4, wd=0.2)

epoch,train_loss,valid_loss,exp_rmspe,time
0,0.007268,0.012054,0.106923,00:06
1,0.007465,0.012356,0.107672,00:06
2,0.007291,0.011856,0.106594,00:06
3,0.007155,0.012197,0.107029,00:06
4,0.006992,0.012017,0.106820,00:07
5,0.006967,0.012135,0.106943,00:06


So now we see that this method produce pretty the same result.

Let's try to incorporate mixup here

### Mixup

And here we just add some interpolation in data (and depended valiable as we have regression here)

In [74]:
alpha = 0.2

In [75]:
def interp(var1, var2, alpha):
    lam = np.random.beta(alpha, alpha)
    return lam*var1 + (1.-lam)*var2

In [76]:
inner_val_idx

range(240858, 253301)

In [77]:
inner_tr_idx = list_diff(list1=range(len(merge_inner_df)), list2=inner_val_idx)

In [78]:
np_merge_df = merge_inner_df.iloc[inner_tr_idx].to_numpy()

In [79]:
augmented = []

In [80]:
for _ in range(6):
    shfld = np_merge_df.copy()
    np.random.shuffle(shfld)
    augmented.append(pd.DataFrame(interp(shfld, np_merge_df, alpha)))

In [83]:
del np_merge_df; del shfld

In [84]:
augmented = pd.concat(augmented)

In [85]:
augmented.rename(columns={augmented.columns[-1]:dep_var}, inplace = True)

In [86]:
len(augmented)

1445148

In [87]:
merge_inner_df = pd.concat([augmented, val_inner_df])

In [88]:
inner_val_idx = range(len(augmented), len(merge_inner_df))

In [89]:
del augmented

In [90]:
inner_cont_vars = list_diff(merge_inner_df.columns, [dep_var])

In [91]:
inner_data = (TabularList.from_df(merge_inner_df, path=path, cat_names=[], cont_names=inner_cont_vars, procs=inner_procs)
             .split_by_idx(inner_val_idx)
             .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
             .databunch(bs=BS))

In [92]:
np.random.seed(1001)
inner_learn = tabular_learner(inner_data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

In [93]:
inner_learn.fit_one_cycle(6, 1e-2, wd=0.2)

epoch,train_loss,valid_loss,exp_rmspe,time
0,0.013896,0.017968,0.141169,00:34
1,0.012299,0.015937,0.120862,00:35
2,0.010756,0.015844,0.127601,00:35
3,0.009529,0.013752,0.114903,00:34
4,0.006879,0.011683,0.109530,00:34
5,0.005538,0.011651,0.107221,00:34


In [94]:
inner_learn.fit_one_cycle(6, 1e-4, wd=0.2)

epoch,train_loss,valid_loss,exp_rmspe,time
0,0.005624,0.011775,0.108276,00:33
1,0.005547,0.011859,0.108505,00:38
2,0.005325,0.012078,0.108924,00:34
3,0.005032,0.012131,0.110331,00:34
4,0.005008,0.012284,0.110623,00:35
5,0.004999,0.012265,0.110361,00:34


So here we don't see any improvments in terms of validation error :(